In [1]:
# importing the libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# importing the datasets.
# i have already downloaded the datasets from the following links:
# https://www.kaggle.com/datasets/kapturovalexander/ign-games-from-best-to-worst/data
# https://www.kaggle.com/datasets/hibrahimag1/top-1000-twitch-streamers-data-may-2024
# https://www.kaggle.com/datasets/rankirsh/evolution-of-top-games-on-twitch

ratings = pd.read_csv(r'C:\!Projects\games_analysis\Data\IGN_games_ratings.csv')
ratings_data = ratings.copy()
streamers = pd.read_csv(r'C:\!Projects\games_analysis\Data\Twitch_streamers_data.csv')
streamers_data = streamers.copy()
games = pd.read_csv(r'C:\!Projects\games_analysis\Data\twitch_games_data.csv', encoding='latin1')
games_data = games.copy()

# - quick view of the datasets

In [3]:
ratings_data.head()

,title,score,score_phrase,platform,genre,release_year,release_month,release_day
0,Checkered Flag,10.0,Masterpiece,Lynx,Racing,1999,7,6
1,Chrono Trigger,10.0,Masterpiece,Wii,"Action, RPG",2011,5,25
2,Dragon Warrior III,10.0,Masterpiece,Game Boy Color,RPG,2001,7,20
3,Grand Theft Auto IV,10.0,Masterpiece,Xbox 360,"Action, Adventure",2008,4,25
4,Grand Theft Auto IV,10.0,Masterpiece,PlayStation 3,"Action, Adventure",2008,4,25


In [4]:
streamers_data.head()

,RANK,NAME,LANGUAGE,TYPE,MOST_STREAMED_GAME,2ND_MOST_STREAMED_GAME,AVERAGE_STREAM_DURATION,FOLLOWERS_GAINED_PER_STREAM,AVG_VIEWERS_PER_STREAM,AVG_GAMES_PER_STREAM,TOTAL_TIME_STREAMED,TOTAL_FOLLOWERS,TOTAL_VIEWS,TOTAL_GAMES_STREAMED,ACTIVE_DAYS_PER_WEEK,MOST_ACTIVE_DAY,DAY_WITH_MOST_FOLLOWERS_GAINED
0,1,kaicenat,English,personality,Just Chatting,I'm Only Sleeping,7.6,18.405,15.852,2.3,4.698,10600000,9150000,194,3.6,Friday,Saturday
1,2,jynxzi,English,personality,Tom Clancy's Rainbow Six Siege,NBA 2K20,5.4,3.386,1.145,1.2,8.407,5760000,1950000,54,5.6,Tuesday,Sunday
2,3,caedrel,English,personality,League of Legends,I'm Only Sleeping,6.3,689.000,12.331,1.3,6.728,797000,14200000,111,2.8,Thursday,Sunday
3,4,caseoh_,English,personality,NBA 2K23,Just Chatting,4.6,7.185,0.000,3.6,2.554,4220000,53,385,6.2,Friday,Monday
4,5,ibai,Spanish,personality,Just Chatting,League of Legends,4.1,8.289,190.714,1.5,6.865,15600000,359000000,149,4.3,Wednesday,Saturday


In [5]:
games_data.head()

,Rank,Game,Month,Year,Hours_watched,Hours_streamed,Peak_viewers,Peak_channels,Streamers,Avg_viewers,Avg_channels,Avg_viewer_ratio
0,1,League of Legends,1,2016,94377226,1362044,530270,2903,129172,127021,1833,69.29
1,2,Counter-Strike: Global Offensive,1,2016,47832863,830105,372654,2197,120849,64378,1117,57.62
2,3,Dota 2,1,2016,45185893,433397,315083,1100,44074,60815,583,104.26
3,4,Hearthstone,1,2016,39936159,235903,131357,517,36170,53749,317,169.29
4,5,Call of Duty: Black Ops III,1,2016,16153057,1151578,71639,3620,214054,21740,1549,14.03


# - data preproccesing

In [14]:
# checking for duplicated rows and missings
# starting with ratings_data
ratings_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18577 entries, 0 to 18624
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   title          18577 non-null  object 
 1   score          18577 non-null  float64
 2   score_phrase   18577 non-null  object 
 3   platform       18577 non-null  object 
 4   genre          18577 non-null  object 
 5   release_year   18577 non-null  int64  
 6   release_month  18577 non-null  int64  
 7   release_day    18577 non-null  int64  
dtypes: float64(1), int64(3), object(4)
memory usage: 1.3+ MB


In [7]:
ratings_data.isnull().sum()

title             0
score             0
score_phrase      0
platform          0
genre            36
release_year      0
release_month     0
release_day       0
dtype: int64

In [8]:
ratings_data['genre'].unique()

array(['Racing', 'Action, RPG', 'RPG', 'Action, Adventure', 'Action',
       'Fighting, Action', 'Adventure', 'Puzzle, Adventure',
       'Sports, Action', 'Fighting', 'Puzzle', 'Platformer', 'Strategy',
       'Sports', 'Racing, Simulation', 'Shooter', 'Compilation',
       'Racing, Action', 'Simulation', 'Strategy, RPG', 'Music',
       'Puzzle, Action', 'Party', 'Music, Action', 'Sports, Simulation',
       'Compilation, Compilation', 'Card', 'Flight, Action', 'Hardware',
       'Shooter, RPG', 'Action, Simulation', 'Wrestling',
       'Platformer, Action', 'Puzzle, RPG', 'Trivia', 'Board',
       'Productivity', nan, 'Action, Compilation', 'Card, Battle',
       'Music, Editor', 'Sports, Racing', 'Flight, Simulation', 'Other',
       'Platformer, Adventure', 'Adventure, Compilation', 'Other, Action',
       'Flight', 'Card, RPG', 'Pinball', 'Educational, Action',
       'RPG, Simulation', 'Adventure, Episodic', 'Shooter, First-Person',
       'Music, Adventure', 'Casino', 'Virtual 

In [9]:
ratings_data.fillna('Not known', inplace=True)
ratings_data.isnull().sum()

title            0
score            0
score_phrase     0
platform         0
genre            0
release_year     0
release_month    0
release_day      0
dtype: int64

In [10]:
ratings_data.tail(20)

,title,score,score_phrase,platform,genre,release_year,release_month,release_day
18605,Kidz Sports: Basketball,1.0,Unbearable,Wii,Sports,2008,2,27
18606,Kidz Sports: Ice Hockey,1.0,Unbearable,Wii,Sports,2008,2,26
18607,Kidz Sports: International Soccer,1.0,Unbearable,Wii,Sports,2008,2,29
18608,London Taxi: Rush Hour,1.0,Unbearable,Wii,Racing,2008,2,29
18609,Mega Man 2,1.0,Unbearable,iPhone,Action,2009,3,27
18610,Monster Trux Arenas -- Special Edition,1.0,Unbearable,Wii,Racing,2008,2,6
18611,Motocross Championship,1.0,Unbearable,Sega 32X,Racing,2008,11,20
18612,NFL Blitz 20-03,1.0,Unbearable,Game Boy Advance,"Sports, Action",2002,8,13
18613,NRA Varmint Hunter,1.0,Unbearable,PC,Shooter,2005,1,28
18614,Offroad Extreme -- Special Edition,1.0,Unbearable,Wii,Racing,2007,12,10


In [11]:
# checking for duplicates
duplicated_rows = ratings_data[ratings_data.duplicated(keep=False)]
duplicated_rows

,title,score,score_phrase,platform,genre,release_year,release_month,release_day
173,Call of Duty: Modern Warfare 2 (Hardened Edition),9.5,Amazing,PlayStation 3,Shooter,2009,11,10
174,Call of Duty: Modern Warfare 2 (Hardened Edition),9.5,Amazing,PlayStation 3,Shooter,2009,11,10
1660,Steel Talons,9.0,Amazing,Lynx,"Flight, Action",1999,7,6
1661,Steel Talons,9.0,Amazing,Lynx,"Flight, Action",1999,7,6
1833,WipEout XL,9.0,Amazing,PlayStation,Racing,1996,11,26
...,...,...,...,...,...,...,...,...
18109,Raven Project,3.0,Awful,PlayStation,Action,1996,11,25
18355,Big League Sports,2.3,Painful,Wii,"Sports, Compilation",2008,11,25
18356,Big League Sports,2.3,Painful,Wii,"Sports, Compilation",2008,11,25
18616,Revolution X,1.0,Unbearable,PlayStation,Action,1996,11,25


In [12]:
ratings_data.drop_duplicates(inplace=True)
ratings_data.duplicated().sum()

0

In [20]:
games_data.tail(20)

,Rank,Game,Month,Year,Hours_watched,Hours_streamed,Peak_viewers,Peak_channels,Streamers,Avg_viewers,Avg_channels,Avg_viewer_ratio
18580,181,Tekken 7,9,2023,657171,41068,7285,99,6013,914,57,16.00
18581,182,Heroes of the Storm,9,2023,654911,30631,7682,85,2259,910,42,21.38
18582,183,Town of Salem 2,9,2023,650654,2085,25432,24,532,904,2,312.06
18583,184,Golf It!,9,2023,645254,2706,145616,31,1398,897,3,238.45
18584,185,Terraria,9,2023,633451,77497,4966,210,15383,881,107,8.17
18585,186,Super Smash Bros. Melee,9,2023,625811,12695,33062,57,1827,870,17,49.30
18586,187,Sekiro: Shadows Die Twice,9,2023,614024,48070,15838,130,9731,853,66,12.77
18587,188,NBA 2K23,9,2023,613649,137164,10198,983,41573,853,190,4.47
18588,189,Call of Duty: Mobile,9,2023,597435,127680,5015,335,43487,830,177,4.68
18589,190,I'm on Observation Duty 6,9,2023,581068,4730,24948,30,1249,808,6,122.85


In [17]:
# Using Gemini OpenAI
import google.generativeai as genai
import apikey

key = apikey.load("game_api")
genai.configure(api_key=key)

model = genai.GenerativeModel('gemini-1.5-flash')
response = model.generate_content("Write a story about a magic backpack.")
print(response.text)

Barnaby was never one for adventure. He preferred the comfort of his armchair, a good book, and a steaming mug of cocoa. So, when his eccentric Aunt Beatrice gifted him a dusty, leather backpack, he wasn't exactly thrilled. "It's a magic backpack, Barnaby," she declared, eyes twinkling, "It can take you anywhere you desire!"

Barnaby scoffed, but the backpack was undeniably peculiar. Its leather was smooth as silk, and the worn brass buckle seemed to hum with an almost imperceptible energy. He reluctantly slung it over his shoulder, feeling the weight shift with each step.

That night, Barnaby dreamed of rolling green hills and shimmering waterfalls. He woke up with a start, a strange sense of longing in his heart. He looked at the backpack, a spark of curiosity flickering in his eyes. Perhaps Aunt Beatrice was right. Perhaps this backpack could be more than just a dusty relic.

He opened the backpack, expecting nothing more than a musty smell and cobwebs. Instead, a bright light erupt